**Parallelism**

Your laptop probably has multiple cores (4 is a typical number). Practically speaking, this means you have the capability of running processes in parallel, i.e. simultaneously. This is especially helpful when the task at hand is trivially parallelizable. 

The kinds of Monte-Carlo simulations we've been doing are great examples. Instead of generating realizations and checking some condition in series we can hand the task off to each processor. Thus, putting aside the overhead associated with making this all work, if we have 4 cores we can do our processing 4 times as fast.

We illustrate with the following question. For a sequence of n Bernoulli trials with success probability p, how many times do we expect to have a 1 followed by a 0 or a 0 followed by a 1?

First consider the non-parallelized version of the code. The code has been written in such a way as to be easily modified when we code the parallel version.

In [ ]:
#
# Non parallel version of Monte-Carlo program.
# Question - Flip a coin P(heads)=p P(tails)=1-p n times. 
# What is the expected number of times we see a change from 0 to 1 or 1 to 0
# We do N Monte-Carlo trials.
#
import numpy as np
import math
import time

#
# Function to return the number of changes from 0 to 1 or 1 to 0
# in a list of 0's and 1's
#
def number_of_changes(X):
    s=X[0] # value in current run
    nchanges=0
    for i in range(1,n):
        if X[i]==s: # no change
            continue
        else:
            s=X[i]
            nchanges+=1
    return(nchanges)

#
# Function to do a single realization of n Bernoulli(p) trials 
# and call the number of changes program. The function takes a tuple 
# of arguments (n,p).
#
def simulate_once(args):
    n=args[0]
    p=args[1]
    X=np.random.choice([0,1],size=n,p=[p,1-p])
    LR=number_of_changes(X)
    return(LR)
#
# Set the values of the parameters
#
N=10000
n=1000
p=.75
#
# Create a list of 2-tuples and apply simulate_once() to every 2-tuple using
# map.
#
time0 = time.process_time()
arglist=[(n,p) for i in range(N)]
result = list(map(simulate_once,arglist))
time1=time.process_time()
print("time = "+ "{:10.3f}".format(time1-time0))
print("mean = " + "{:10.3f}".format(np.mean(np.array(result))))
print("std err = " + "{:10.3f}".format(np.std(np.array(result))/np.sqrt(N)))


**Parallel version**

Here is the parallel version of the progrm (which doesn't work in jupyter notebooks). To run this program, copy it to a file - e.g. program2.py in some folder e in your file system, then in a terminal (or anaconda prompt) navigate to that folder and at the prompt (shown here as >), (assuming python is in your path) type:

$>$ python program.py

Alternatively, you can use the spyder IDE or some other environment to run the code.


In [ ]:
import numpy as np
import math
import multiprocessing as mp
import time

#
# Function to return the number of changes from 0 to 1 or 1 to 0
# in a list of 0's and 1's
#
def number_of_changes(X):
    s=X[0] # value in current run
    nchanges=0
    for i in range(1,n):
        if X[i]==s: # no change
            continue
        else:
            s=X[i]
            nchanges+=1
    return(nchanges)


def simulate_once(args):
    n=args[0]
    p=args[1]
    X=np.random.choice([0,1],size=n,p=[p,1-p])
    LR=number_of_changes(X)
    return(LR)

N=1000
n=1000
p=.75
nprocesses=4


if __name__=="__main__":
    time0 = time.time()
    #
    # create a pool of workers
    #
    pool = mp.Pool(nprocesses)
    #
    # create a list of arguments
    #
    arglist=[(n,p) for i in range(N)]
    #
    # use pool.map to apply the simulate_once function to every
    # element in the argument list
    #
    result = pool.map(simulate_once,arglist)
    pool.close()
    pool.join()
    time1=time.time()
    print("time = "+ "{:10.3f}".format(time1-time0))
    print("mean = " + "{:10.3f}".format(np.mean(np.array(result))))
    print("std err = " + "{:10.3f}".format(np.std(np.array(result))/np.sqrt(N)))


In [ ]:
4

The following is from stackoverflow: https://stackoverflow.com/questions/20360686/compulsory-usage-of-if-name-main-in-windows-while-using-multiprocessi

The multiprocessing module works by creating new Python processes that will import your module. If you did not add __name__== '__main__' protection then you would enter a never ending loop of new process creation. It goes like this:

- Your module is imported and executes code during the import that cause multiprocessing to spawn 4 new processes.

- Those 4 new processes in turn import the module and executes code during the import that cause multiprocessing to spawn 16 new processes.

- Those 16 new processes in turn import the module and executes code during the import that cause multiprocessing to spawn 64 new processes.

Well, hopefully you get the picture.

So the idea is that you make sure that the process spawning only happens once. And that is achieved most easily with the idiom of the __name__== '__main__' protection.